# Mergen categorieën

In [3]:
#imports
import pandas as pd
import datetime as dt
import numpy as np
import itertools
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
app_filename = "C:/Users/fdnecker/OneDrive - UGent/UGent/2019-2020/Doctoraat Floor/2019-2020/Smartphone_slaap_SimonA/MobileDNA data/input/siam1_appevents.csv"

In [8]:
app_filename

'C:/Users/fdnecker/OneDrive - UGent/UGent/2019-2020/Doctoraat Floor/2019-2020/Smartphone_slaap_SimonA/MobileDNA data/input/siam1_appevents.csv'

In [9]:
df = pd.read_csv(app_filename, sep=";", parse_dates=['startTime', 'endTime'])
df.drop("Unnamed: 0", axis=1, inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234061 entries, 0 to 234060
Data columns (total 15 columns):
application        234061 non-null object
battery            234061 non-null float64
data_version       234061 non-null float64
endTime            234061 non-null datetime64[ns]
endTimeMillis      234061 non-null int64
id                 234061 non-null object
latitude           234061 non-null float64
longitude          234061 non-null float64
model              234061 non-null object
notification       234061 non-null bool
session            234061 non-null int64
startTime          234061 non-null datetime64[ns]
startTimeMillis    234061 non-null int64
studyKey           234061 non-null object
surveyId           234061 non-null object
dtypes: bool(1), datetime64[ns](2), float64(4), int64(3), object(5)
memory usage: 25.2+ MB


In [10]:
#drop duplicates
pd.concat(g for _, g in df.groupby(["startTime","application","endTime"]) if len(g) > 1)

df = df.drop_duplicates(["startTime","application","endTime"])

In [11]:
#develop necessary variables
df['startTime'] = pd.to_datetime(df['startTime'])
df['endTime'] = pd.to_datetime(df['endTime'])
df['dag'] = df.startTime.dt.date
df['uur'] = df.startTime.dt.hour

In [12]:
#duur variable
df['duur'] = df['endTimeMillis'] - df['startTimeMillis']
df['duur_sec'] = df['duur']/1000
df['duur_min'] = df['duur_sec']/60

In [13]:
df.head()

,application,battery,data_version,endTime,endTimeMillis,id,latitude,longitude,model,notification,session,startTime,startTimeMillis,studyKey,surveyId,dag,uur,duur,duur_sec,duur_min
0,be.ugent.mobiledna,94.0,1.5,2018-12-27 12:13:44.834,1545909224834,a011f110-bd7a-49ca-8e0c-a3368e84cf12,51.184188,3.740566,SM-A510F,False,1545909198,2018-12-27 12:13:42.433,1545909222433,SIAM1,28042000lijiki,2018-12-27,12,2401,2.401,0.040017
1,com.sec.android.app.sbrowser,94.0,1.5,2018-12-27 12:14:40.919,1545909280919,a011f110-bd7a-49ca-8e0c-a3368e84cf12,51.184188,3.740566,SM-A510F,False,1545909198,2018-12-27 12:13:44.848,1545909224848,SIAM1,28042000lijiki,2018-12-27,12,56071,56.071,0.934517
2,com.android.systemui,87.0,1.5,2018-12-27 13:59:19.398,1545915559398,a011f110-bd7a-49ca-8e0c-a3368e84cf12,51.184110,3.740284,SM-A510F,False,1545915327,2018-12-27 13:59:18.019,1545915558019,SIAM1,28042000lijiki,2018-12-27,13,1379,1.379,0.022983
3,be.argenta.bankieren,87.0,1.5,2018-12-27 13:59:38.957,1545915578957,a011f110-bd7a-49ca-8e0c-a3368e84cf12,51.184110,3.740284,SM-A510F,False,1545915327,2018-12-27 13:59:19.432,1545915559432,SIAM1,28042000lijiki,2018-12-27,13,19525,19.525,0.325417
4,com.android.systemui,87.0,1.5,2018-12-27 14:01:00.165,1545915660165,a011f110-bd7a-49ca-8e0c-a3368e84cf12,51.184110,3.740284,SM-A510F,False,1545915327,2018-12-27 14:00:59.274,1545915659274,SIAM1,28042000lijiki,2018-12-27,14,891,0.891,0.014850


In [ ]:
#de specifieke appevents linken met categorie. 

In [14]:
cat = pd.read_excel('C:/Users/fdnecker/OneDrive - UGent/UGent/2019-2020/Doctoraat Floor/2019-2020/Smartphone_slaap_SimonA/MobileDNA data/input/191217_app_categories.xlsx')

In [19]:
cat.head()

,appname,fancyname,category_new,category,company,extrainfo,source,purchases,ads,sharesinfo,shareslocation,unrestrictedinternet,usersinteract,game,Rating,Parentalguidance,Downloads
0,com.netflix.mediaclient,Netflix,video,entertainment,"Netflix, Inc.","offers in-app purchases \ august 19, 2019 \ va...",google play,1,0,0,0,0,0,0.0,No Info,1.0,"500,000,000+"
1,com.facebook.orca,Messenger – Text and Video Chat for Free,NaN,communication,Facebook,"offers in-app purchases \ august 20, 2019 \ va...",google play,1,0,0,1,0,1,0.0,everyone,0.0,"1,000,000,000+"
2,com.android.chrome,Google Chrome: Fast & Secure,NaN,communication,Google LLC,"august 14, 2019 \ varies with device \ 5,000,0...",google play,0,0,0,0,1,0,0.0,everyone,0.0,"5,000,000+"
3,com.google.android.gm,Gmail,NaN,communication,Google LLC,"contains ads \ august 19, 2019 \ varies with d...",google play,0,1,0,0,0,1,0.0,everyone,0.0,"5,000,000+"
4,com.grindrapp.android,Grindr - Gay chat,NaN,social,Grindr LLC,contains ads·offers in-app purchases \ august ...,google play,1,1,0,0,0,1,0.0,18+,0.0,"10,000,000+"


In [20]:
basis_df = pd.merge(
df, 
cat[["appname", "fancyname", "category"]], 
left_on="application", 
right_on="appname", 
how = "left"
)

In [21]:
basis_df.head()

,application,battery,data_version,endTime,endTimeMillis,id,latitude,longitude,model,notification,...,studyKey,surveyId,dag,uur,duur,duur_sec,duur_min,appname,fancyname,category
0,be.ugent.mobiledna,94.0,1.5,2018-12-27 12:13:44.834,1545909224834,a011f110-bd7a-49ca-8e0c-a3368e84cf12,51.184188,3.740566,SM-A510F,False,...,SIAM1,28042000lijiki,2018-12-27,12,2401,2.401,0.040017,be.ugent.mobiledna,mobileDNA,lifestyle
1,com.sec.android.app.sbrowser,94.0,1.5,2018-12-27 12:14:40.919,1545909280919,a011f110-bd7a-49ca-8e0c-a3368e84cf12,51.184188,3.740566,SM-A510F,False,...,SIAM1,28042000lijiki,2018-12-27,12,56071,56.071,0.934517,com.sec.android.app.sbrowser,Samsung Internet Browser,communication
2,com.android.systemui,87.0,1.5,2018-12-27 13:59:19.398,1545915559398,a011f110-bd7a-49ca-8e0c-a3368e84cf12,51.184110,3.740284,SM-A510F,False,...,SIAM1,28042000lijiki,2018-12-27,13,1379,1.379,0.022983,com.android.systemui,NaN,system
3,be.argenta.bankieren,87.0,1.5,2018-12-27 13:59:38.957,1545915578957,a011f110-bd7a-49ca-8e0c-a3368e84cf12,51.184110,3.740284,SM-A510F,False,...,SIAM1,28042000lijiki,2018-12-27,13,19525,19.525,0.325417,be.argenta.bankieren,Argenta Banking,finance
4,com.android.systemui,87.0,1.5,2018-12-27 14:01:00.165,1545915660165,a011f110-bd7a-49ca-8e0c-a3368e84cf12,51.184110,3.740284,SM-A510F,False,...,SIAM1,28042000lijiki,2018-12-27,14,891,0.891,0.014850,com.android.systemui,NaN,system


## Avondgebruik (tussen 21u en 24)

In [32]:
def avondgebruik(row):
    uur = row["uur"]
    
    if (uur >= 6) & (uur < 21):
        return "overdag gebruik"
    elif (uur >= 21) & (uur <= 23):
        return "avondgebruik"
    elif (uur >= 0) & (uur < 6):
        return "nachtgebruik"
    else:
        return "niet gevonden"

In [33]:
basis_df['gebruik'] = basis_df.apply(avondgebruik, axis=1)

In [34]:
basis_df.head()

,application,battery,data_version,endTime,endTimeMillis,id,latitude,longitude,model,notification,...,surveyId,dag,uur,duur,duur_sec,duur_min,appname,fancyname,category,gebruik
0,be.ugent.mobiledna,94.0,1.5,2018-12-27 12:13:44.834,1545909224834,a011f110-bd7a-49ca-8e0c-a3368e84cf12,51.184188,3.740566,SM-A510F,False,...,28042000lijiki,2018-12-27,12,2401,2.401,0.040017,be.ugent.mobiledna,mobileDNA,lifestyle,overdag gebruik
1,com.sec.android.app.sbrowser,94.0,1.5,2018-12-27 12:14:40.919,1545909280919,a011f110-bd7a-49ca-8e0c-a3368e84cf12,51.184188,3.740566,SM-A510F,False,...,28042000lijiki,2018-12-27,12,56071,56.071,0.934517,com.sec.android.app.sbrowser,Samsung Internet Browser,communication,overdag gebruik
2,com.android.systemui,87.0,1.5,2018-12-27 13:59:19.398,1545915559398,a011f110-bd7a-49ca-8e0c-a3368e84cf12,51.184110,3.740284,SM-A510F,False,...,28042000lijiki,2018-12-27,13,1379,1.379,0.022983,com.android.systemui,NaN,system,overdag gebruik
3,be.argenta.bankieren,87.0,1.5,2018-12-27 13:59:38.957,1545915578957,a011f110-bd7a-49ca-8e0c-a3368e84cf12,51.184110,3.740284,SM-A510F,False,...,28042000lijiki,2018-12-27,13,19525,19.525,0.325417,be.argenta.bankieren,Argenta Banking,finance,overdag gebruik
4,com.android.systemui,87.0,1.5,2018-12-27 14:01:00.165,1545915660165,a011f110-bd7a-49ca-8e0c-a3368e84cf12,51.184110,3.740284,SM-A510F,False,...,28042000lijiki,2018-12-27,14,891,0.891,0.014850,com.android.systemui,NaN,system,overdag gebruik


In [35]:
basis_df.to_excel('basis_df.xlsx')